In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
import skimage, os
from skimage.morphology import ball, disk, dilation,binary_dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing, binary_opening 
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border, mark_boundaries
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
from glob import glob
from skimage.io import imread


In [3]:
covidx_dir = '/media/ubuntu/MyHDataStor3/datasets/COV19D/'
covidx_img_dir= covidx_dir 
covidx_mask_dir= covidx_dir + 'mask/' 
data_list_dir = '/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/'

print(covidx_dir)
print(covidx_img_dir)
print(covidx_mask_dir)
print(data_list_dir)

/media/ubuntu/MyHDataStor3/datasets/COV19D/
/media/ubuntu/MyHDataStor3/datasets/COV19D/
/media/ubuntu/MyHDataStor3/datasets/COV19D/mask/
/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/


In [4]:
def seedfill(img):

  rows,cols = img.shape 

  img_floodfill = img.copy()
  mask = np.zeros((rows+2,cols+2), np.uint8)

  '''
  # Floodfill from point (0, 0)
  found  = False 
  for x in range(cols): 
    for y in range(rows):
      if img_floodfill[y,x] == 0:
        found = True 
        break  
    if found:
      break 

  #cv2.floodFill(img_floodfill, mask, (x,y), 255);
  ''' 
  

  cv2.floodFill(img_floodfill, mask, (0,0), 255);
  cv2.floodFill(img_floodfill, mask, (0,rows-1), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,0), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,rows-1), 255);

  return img_floodfill 


In [5]:
def body_contour(binary_image):
    """Helper function to get body contour"""
    contours, hier = cv2.findContours(binary_image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    body_idx = np.argmax(areas)
    return contours[body_idx]


def auto_body_crop(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(3, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    #print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)         
    
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
    
    
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    '''
    #find the lung rectangle        
    binary2 = ((1-bin_image)*255).astype(np.uint8) 
    kernel = np.ones((5,5), np.uint8) 
    binary2 = cv2.dilate(binary2, kernel, iterations=2) 
    binary2 = cv2.erode(binary2, kernel, iterations=2) 
    binary2 = seedfill(binary2) 
    binary2 = cv2.bitwise_not(binary2)    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(binary2, cmap=plt.cm.bone) 
    '''    
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
    return bin_image, (xmin, ymin, xmax, ymax)


def auto_segment(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(9, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)             
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
        
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
 
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
        
    bin_image_inv = (1 - bin_image)*255 
    bin_image_inv = seedfill(bin_image_inv)/255 
    cleared = clear_border(bin_image_inv)
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(cleared, cmap=plt.cm.bone)     
    
    label_image = label(cleared)
    if plot == True:
        plots[4].axis('off')
        plots[4].imshow(label_image, cmap=plt.cm.bone) 
        
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0

    binary = label_image > 0
    if plot == True:
        plots[5].axis('off')
        plots[5].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(7)
    binary = binary_erosion(binary, selem)
    if plot == True:
        plots[6].axis('off')
        plots[6].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot == True:
        plots[7].axis('off')
        plots[7].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''

    binary1 = binary.copy() 

    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    if plot == True:
        plots[8].axis('off')
        plots[8].imshow(binary, cmap=plt.cm.bone) 
    
    
    return binary, binary1, (xmin, ymin, xmax, ymax)


In [6]:
def refine_mask(m_full):
    
    mask2 = cv2.imread(m_full, cv2.IMREAD_UNCHANGED)
    mask2 = (mask2/255).astype(np.uint8) 
    
    selem = disk(10)
    mask2_2  = binary_closing(mask2, selem)                
    label_image = label(mask2_2)
    
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                    label_image[coordinates[0], coordinates[1]] = 0
    mask2_3 = label_image > 0               
    
    edges = roberts(mask2_3) 
    mask2_4 = ndi.binary_fill_holes(edges)
                
    return mask2,mask2_4

In [8]:
#subsets = ['val','train']  #for test dataset, UNet segmentation and refinement are combined. 
subsets = ['train']

from tqdm import tqdm

debug = False
split = 1 

for subset in subsets:     

    list_file = data_list_dir + '{}_rgb_split{}.txt'.format(subset,split) 
    lines = open(list_file,"r").readlines() 

    pbar = tqdm(total = len(lines))
    
    for ind in range(len(lines)-1,-1,-2):
        
        l = lines[ind]
       
        print(l)
        
        pbar.update()
        
        l_splits = l.split()  
                
        l_path = l_splits[0]        
        c = l_splits[1]
        first_slice = int(l_splits[2])
        last_slice = int(l_splits[3])

        count = 0 
        for f in range(first_slice,last_slice+1):
            
            count +=1 
            
            #if count>10:
            #    break 
            
            fn = "{}/{}.jpg".format(l_path,f)
            f_full = covidx_img_dir + fn
            m_full = covidx_img_dir + 'mask/' + fn
            m_full_new = m_full+'_refined.jpg'            
            #print(f_full,m_full,m_full_new)
            
            if not os.path.exists(m_full_new) :
                mask0,mask = refine_mask(m_full)                
                cv2.imwrite(m_full_new, (mask*255).astype(np.uint8) )

                if False: 
                    fig, (ax1,ax2) = plt.subplots(1,2)
                    ax1.imshow(mask0*255,cmap='gray')
                    ax2.imshow(mask*255,cmap='gray')                
                



 10%|▉         | 198/1992 [00:00<00:00, 1977.33it/s]

train/covid/subset3/ct_scan_498 covid 22 69 0 73 512 419

train/covid/subset2/ct_scan_332 covid 19 35 0 117 497 436

train/covid/subset4/ct_scan842 covid 12 26 15 112 506 440

train/covid/subset3/ct_scan_490 covid 100 187 0 28 497 410

train/covid/subset4/ct_scan850 covid 12 29 10 67 500 429

train/covid/subset1/ct_scan_38 covid 120 327 0 100 512 470

train/covid/subset1/ct_scan_223 covid 21 36 25 112 480 414

train/covid/subset3/ct_scan_584 covid 34 70 4 88 505 432

train/covid/subset4/ct_scan741 covid 63 235 0 107 512 415

train/covid/subset1/ct_scan_29 covid 12 34 0 110 512 458

train/covid/subset1/ct_scan_164 covid 20 39 0 107 512 435

train/covid/subset1/ct_scan_131 covid 14 27 0 110 512 460

train/covid/subset3/ct_scan_475 covid 93 179 25 152 487 434

train/covid/subset2/ct_scan_436 covid 102 189 0 133 512 470

train/covid/subset3/ct_scan_561 covid 28 75 38 97 499 444

train/covid/subset3/ct_scan_614 covid 88 282 5 104 502 412

train/covid/subset4/ct_scan751 covid 113 244 0 107 5


 17%|█▋        | 338/1992 [00:04<00:16, 97.92it/s]  

train/covid/subset4/ct_scan797 covid 123 265 13 77 491 391

train/covid/subset2/ct_scan_376 covid 26 45 93 182 492 427

train/covid/subset2/ct_scan_239 covid 22 48 11 94 495 422




 17%|█▋        | 341/1992 [00:21<45:39,  1.66s/it]

...


 29%|██▉       | 580/1992 [06:18<2:11:59,  5.61s/it]


train/covid/subset3/ct_scan_628 covid 66 208 0 73 512 419

train/covid/subset1/ct_scan_50 covid 81 192 0 109 512 474

train/covid/subset3/ct_scan_560 covid 69 82 0 46 506 384

train/covid/subset4/ct_scan815 covid 8 27 34 100 481 414

train/covid/subset4/ct_scan725 covid 82 180 13 79 491 392

train/covid/subset2/ct_scan_440 covid 20 71 0 76 512 426

train/covid/subset1/ct_scan_142 covid 16 36 0 65 512 495

train/covid/subset3/ct_scan_537 covid 28 85 0 79 512 424

train/covid/subset4/ct_scan757 covid 104 198 0 57 512 407

train/covid/subset1/ct_scan_40 covid 101 218 0 85 512 471

train/covid/subset3/ct_scan_675 covid 10 31 0 115 512 405

train/covid/subset2/ct_scan_325 covid 122 296 0 63 512 452

train/covid/subset1/ct_scan_99 covid 78 211 0 9 512 512

train/covid/subset4/ct_scan706 covid 57 204 0 48 512 463

train/covid/subset1/ct_scan_85 covid 122 228 0 107 512 437

train/covid/subset1/ct_scan_135 covid 103 138 0 55 512 412

train/covid/subset3/ct_scan_643 covid 19 64 0 55 508 441

tra

 47%|████▋     | 935/1992 [06:18<48:25,  2.75s/it]  

train/non-covid/subset4/ct_scan_680 non-covid 91 243 71 134 448 396

train/non-covid/subset1/ct_scan_288 non-covid 74 210 41 126 481 452

train/non-covid/subset5/ct_scan1002 non-covid 127 253 0 89 512 512

train/non-covid/subset1/ct_scan_293 non-covid 148 352 26 82 512 430

train/non-covid/subset5/ct_scan1066 non-covid 27 68 0 38 512 427

train/non-covid/subset5/ct_scan1063 non-covid 32 74 41 90 475 391

train/non-covid/subset3/ct_scan_35 non-covid 40 47 103 189 505 410

train/non-covid/subset4/ct_scan_629 non-covid 22 51 13 30 502 343

train/non-covid/subset3/ct_scan_70 non-covid 107 152 11 123 486 442

train/non-covid/subset3/ct_scan_159 non-covid 80 218 5 113 512 443

train/non-covid/subset4/ct_scan_726 non-covid 85 191 0 76 512 389

train/non-covid/subset2/ct_scan_367 non-covid 123 379 0 116 512 426

train/non-covid/subset4/ct_scan_624 non-covid 15 33 13 96 499 417

train/non-covid/subset5/ct_scan920 non-covid 106 222 0 91 508 463

train/non-covid/subset4/ct_scan_492 non-covid 12 2


 50%|█████     | 996/1992 [06:31<45:37,  2.75s/it]